In [1]:
import numpy as np
from numpy.random import permutation
import keras
from keras.utils import np_utils
from keras.models import Model, Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.optimizers import SGD, RMSprop
from keras import backend
from keras.layers import Input, merge
from keras.layers.core import Dense, Lambda, Reshape
from keras.layers.convolutional import Conv1D
from keras.models import Model

Using TensorFlow backend.


In [2]:
batch_size = 128
nb_epoch = 200
J = 0
#J = 4 #nb of negative targets
L = 32 #for last Dense layers

In [24]:
# For training
genes_1 = np.load('input10k-7gram_gene_seed8.npy') # 100 000 pairs from the head of "input_for_neuronet
mirs_1 = np.load('input10k-7gram_mir_seed8.npy')
genes_0 = np.load('control10k-7gram_gene_seed8.npy') # 100 000 pairs from the head of "input_for_neuronet
mirs_0 = np.load('control10k-7gram_mir_seed8.npy')

In [1]:
int(16.0)

16


# Generation of 4 'negative' genes by simply shuffling the original genes' set
perm = permutation(len(genes_1))
genes_01 = genes_1[perm]
print perm, genes_01
perm = permutation(len(genes_1))
genes_02 = genes_1[perm]
perm = permutation(len(genes_1))
genes_03 = genes_1[perm]
perm = permutation(len(genes_1))
genes_04 = genes_1[perm]

In [37]:
genes = np.concatenate((genes_1, genes_0))
                        #, genes_02, genes_03
                        #, genes_04))
targets_1 = np.empty(genes_1.shape[0])
targets_1.fill(1) 
targets_0 = np.zeros(genes_1.shape[0])
targets = np.concatenate((targets_1, targets_0))
mirs = np.concatenate((mirs_1, mirs_0))
                       #, mirs_1, mirs_1
                       #, mirs_1))

perm = permutation(len(genes))
genes = genes[perm]
mirs = mirs[perm]
targets = targets[perm]
# For test
genes_t = genes[9000:-1]
mirs_t = mirs[9000:-1]
targets_t = targets[9000:-1]

genes = genes[0:9000]
mirs = mirs[0:9000]
targets = targets[0:9000]

In [31]:
# For test
genes_t = np.load('input10ktail-7gram_mir_seed8.npy') # 10 000 pairs from the tail
mirs_t = np.load('input10ktail-7gram_gene_seed8.npy')
perm = permutation(len(genes_1t))
genes_01t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_02t = genes_1[perm]
perm = permutation(len(genes_1t))
genes_03t = genes_1[perm]
#perm = permutation(len(genes_1t))
#genes_04t = genes_1[perm]

#genes_t = np.concatenate((genes_1t))
                          #, genes_01t, genes_02t, genes_03t))
                          #, genes_04t))
targets_t = np.empty(genes_1t.shape[0])
targets_t.fill(1) 
targets_0t = np.zeros(genes_1t.shape[0])
#targets_t = np.concatenate((targets_1t))
                            #, targets_0t))
#mirs_t = np.concatenate((mirs_1t))
                         #, mirs_1t, mirs_1t, mirs_1t))
                         #, mirs_1t))

perm_t = permutation(len(genes_t))
genes_t = genes_t[perm_t]
mirs_t = mirs_t[perm_t]
targets_t = targets_t[perm_t]

In [76]:
dense1 = 64
dense2 = 32
dense3 = 1

# targets 
genes_branch = Sequential()
genes_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
genes_branch.add(Dense(dense2))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense3))
#genes_branch.add(Activation('sigmoid'))

# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(dense2))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense3))
#mirs_branch.add(Activation('sigmoid'))

In [77]:
# R_Q_D= the cosine similarity between microRNAs and genes
R_Q_D = Merge([mirs_branch, genes_branch], mode = 'concat') # See equation (4).
model_Rs = Sequential()
#model_Rs.add(concat_Rs)
model_Rs.add(R_Q_D)
model_Rs.add(Dense(1))
model_Rs.add(Activation('sigmoid'))
print model_Rs.output_shape
#model_Rs.add(Reshape((J + 1, 1)))
print model_Rs.output_shape
#weight = np.array([1]).reshape(1, 1, 1,1)
#model_Rs.add(Conv1D(1, 1, border_mode = "same", 
 #                    input_shape = (J + 1, 1), 
#                  activation = "linear", bias = False,
#                       weights = [weight]))
#model_Rs.add(Reshape((J + 1, )))
#print model_Rs.output_shape

(None, 1)
(None, 1)


In [78]:
sgd = SGD(lr=0.00015, decay=1e-6, momentum=0.7, nesterov=True)
#model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics=['accuracy','recall','precision'])

In [79]:
model_Rs.fit([mirs, genes], targets,
             batch_size=batch_size,
              nb_epoch=40,
              validation_split=0.1)
            #shuffle = True)

Train on 8100 samples, validate on 900 samples
Epoch 1/40
8100/8100 [==============================] - 0s - loss: 0.6327 - acc: 0.6474 - recall: 0.7389 - precision: 0.6294 - val_loss: 0.5733 - val_acc: 0.7833 - val_recall: 0.8257 - val_precision: 0.7458
Epoch 2/40
8100/8100 [==============================] - 0s - loss: 0.5700 - acc: 0.7286 - recall: 0.6818 - precision: 0.7525 - val_loss: 0.5242 - val_acc: 0.7944 - val_recall: 0.7378 - val_precision: 0.8131
Epoch 3/40
8100/8100 [==============================] - 0s - loss: 0.5234 - acc: 0.7590 - recall: 0.6908 - precision: 0.7991 - val_loss: 0.4958 - val_acc: 0.8044 - val_recall: 0.7582 - val_precision: 0.8171
Epoch 4/40
8100/8100 [==============================] - 0s - loss: 0.5010 - acc: 0.7760 - recall: 0.7072 - precision: 0.8198 - val_loss: 0.4767 - val_acc: 0.8133 - val_recall: 0.7841 - val_precision: 0.8155
Epoch 5/40
8100/8100 [==============================] - 0s - loss: 0.4789 - acc: 0.7889 - recall: 0.7261 - precision: 0.8297 

In [80]:
# test
model_Rs.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

 9984/10999 [==========================>...] - ETA: 0s

[0.43617190858084176,
 0.86435130467489851,
 0.84643279279792449,
 0.87918453712833611]

In [81]:
# For training
genes_1 = np.load('input100k-7gram_gene_seed8.npy') # 100 000 pairs from the head of "input_for_neuronet
mirs_1 = np.load('input100k-7gram_mir_seed8.npy')
genes_0 = np.load('control100k-7gram_gene_seed8.npy') # 100 000 pairs from the head of "input_for_neuronet
mirs_0 = np.load('control100k-7gram_mir_seed8.npy')

genes = np.concatenate((genes_1, genes_0))
                        #, genes_02, genes_03
                        #, genes_04))
targets_1 = np.empty(genes_1.shape[0])
targets_1.fill(1) 
targets_0 = np.zeros(genes_1.shape[0])
targets = np.concatenate((targets_1, targets_0))
mirs = np.concatenate((mirs_1, mirs_0))
                       #, mirs_1, mirs_1
                       #, mirs_1))

perm = permutation(len(genes))
genes = genes[perm]
mirs = mirs[perm]
targets = targets[perm]
# For test
genes_t = genes[90000:-1]
mirs_t = mirs[90000:-1]
targets_t = targets[90000:-1]

genes = genes[0:90000]
mirs = mirs[0:90000]
targets = targets[0:90000]

In [ ]:
dense1 = 64
dense2 = 32
dense3 = 1

# targets 
genes_branch = Sequential()
genes_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
genes_branch.add(Dense(dense2))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense3))
#genes_branch.add(Activation('sigmoid'))

# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(dense2))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense3))
#mirs_branch.add(Activation('sigmoid'))
R_Q_D = Merge([mirs_branch, genes_branch], mode = 'concat') # See equation (4).
model_Rs = Sequential()
#model_Rs.add(concat_Rs)
model_Rs.add(R_Q_D)
model_Rs.add(Dense(1))
model_Rs.add(Activation('sigmoid'))
sgd = SGD(lr=0.00015, decay=1e-6, momentum=0.7, nesterov=True)
#model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics=['accuracy','recall','precision'])
model_Rs.fit([mirs, genes], targets,
             batch_size=batch_size,
              nb_epoch=20,
              validation_split=0.1)
            #shuffle = True)

In [ ]:
# test
model_Rs.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

In [ ]:
# For training
genes_1 = np.load('input100k-7gram_gene_seed16.npy') # 10 000 pairs from the head of "input_for_neuronet
mirs_1 = np.load('input100k-7gram_mir_seed16.npy')
genes_0 = np.load('control100k-7gram_gene_seed16.npy') # 10 000 pairs from the head of "input_for_neuronet
mirs_0 = np.load('control100k-7gram_mir_seed16.npy')

genes = np.concatenate((genes_1, genes_0, genes_0, genes_0))
                        #, genes_02, genes_03
                        #, genes_04))
targets_1 = np.empty(genes_1.shape[0])
targets_1.fill(1) 
targets_0 = np.zeros(3*genes_1.shape[0])
targets = np.concatenate((targets_1, targets_0))
mirs = np.concatenate((mirs_1, mirs_0, mirs_0, mirs_0))
                       #, mirs_1, mirs_1
                       #, mirs_1))

perm = permutation(len(genes))
genes = genes[perm]
mirs = mirs[perm]
targets = targets[perm]
# For test
train_size = 0.8*len(genes)
genes_t = genes[train_size:-1]
mirs_t = mirs[train_size:-1]
targets_t = targets[train_size:-1]

genes = genes[0:train_size]
mirs = mirs[0:train_size]
targets = targets[0:train_size]

In [5]:
dense1 = 64
dense2 = 32
dense3 = 1

# targets 
genes_branch = Sequential()
genes_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
genes_branch.add(Dense(dense2))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense3))
#genes_branch.add(Activation('sigmoid'))

# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(dense2))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense3))
#mirs_branch.add(Activation('sigmoid'))
R_Q_D = Merge([mirs_branch, genes_branch], mode = 'concat') # See equation (4).
model_Rs = Sequential()
#model_Rs.add(concat_Rs)
model_Rs.add(R_Q_D)
model_Rs.add(Dense(1))
model_Rs.add(Activation('sigmoid'))
sgd = SGD(lr=0.00015, decay=1e-6, momentum=0.7, nesterov=True)
#model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics=['accuracy','recall','precision'])
model_Rs.fit([mirs, genes], targets,
             batch_size=batch_size,
              nb_epoch=20,
              validation_split=0.1)
            #shuffle = True)


Train on 28800 samples, validate on 3200 samples
Epoch 1/20
28800/28800 [==============================] - 2s - loss: 0.3495 - acc: 0.8599 - recall: 0.5477 - precision: 0.8145 - val_loss: 0.1533 - val_acc: 0.9509 - val_recall: 0.8318 - val_precision: 0.9679
Epoch 2/20
28800/28800 [==============================] - 1s - loss: 0.1623 - acc: 0.9440 - recall: 0.8391 - precision: 0.9285 - val_loss: 0.1025 - val_acc: 0.9697 - val_recall: 0.8949 - val_precision: 0.9854
Epoch 3/20
28800/28800 [==============================] - 1s - loss: 0.1073 - acc: 0.9660 - recall: 0.9079 - precision: 0.9536 - val_loss: 0.1188 - val_acc: 0.9694 - val_recall: 0.8848 - val_precision: 0.9943
Epoch 4/20
28800/28800 [==============================] - 1s - loss: 0.0803 - acc: 0.9767 - recall: 0.9387 - precision: 0.9683 - val_loss: 0.0831 - val_acc: 0.9800 - val_recall: 0.9306 - val_precision: 0.9886
Epoch 5/20
28800/28800 [==============================] - 1s - loss: 0.0633 - acc: 0.9839 - recall: 0.9577 - precis

In [6]:
model_Rs.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

7999/7999 [==============================] - 0s     


[0.16960426814961727,
 0.97849731217147207,
 0.91971187983353242,
 0.99563052490631154]

In [15]:
genes_branch = Sequential()
genes_branch.add(Dense(128, input_dim=genes.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))

mirs_branch = Sequential()
mirs_branch.add(Dense(128, input_dim=genes.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))

merged = Merge([genes_branch, mirs_branch], mode='concat') #Multiple Sequential instances can be merged into a 
# single output via a Merge layer. The output is a layer 
model = Sequential()
model.add(merged)
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

batch_size = 256
nb_epoch = 400
sgd = SGD(lr=0.0002, decay=1e-6, momentum=0.8, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit([genes, mirs], targets,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1,
              shuffle=True)

Train on 8100 samples, validate on 900 samples
Epoch 1/400
8100/8100 [==============================] - 1s - loss: 0.8470 - acc: 0.4968 - val_loss: 0.7223 - val_acc: 0.5122
Epoch 2/400
8100/8100 [==============================] - 0s - loss: 0.7959 - acc: 0.5062 - val_loss: 0.6917 - val_acc: 0.5422
Epoch 3/400
8100/8100 [==============================] - 0s - loss: 0.7738 - acc: 0.5165 - val_loss: 0.6745 - val_acc: 0.5556
Epoch 4/400
8100/8100 [==============================] - 0s - loss: 0.7369 - acc: 0.5407 - val_loss: 0.6645 - val_acc: 0.5622
Epoch 5/400
8100/8100 [==============================] - 0s - loss: 0.7232 - acc: 0.5330 - val_loss: 0.6588 - val_acc: 0.5622
Epoch 6/400
8100/8100 [==============================] - 0s - loss: 0.7166 - acc: 0.5358 - val_loss: 0.6549 - val_acc: 0.5622
Epoch 7/400
8100/8100 [==============================] - 0s - loss: 0.7173 - acc: 0.5388 - val_loss: 0.6521 - val_acc: 0.5567
Epoch 8/400
8100/8100 [==============================] - 0s - loss: 0.6

In [16]:
# test
model.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

 9728/10999 [=========================>....] - ETA: 0s

[1.3236495136336768, 0.50350031817823182]

In [7]:
dense1 = 128
dense2 = 32
dense3 = 1

# targets 
genes_branch = Sequential()
genes_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
genes_branch.add(Dense(dense2))
genes_branch.add(Activation('relu'))
genes_branch.add(Dropout(0.5))
#genes_branch.add(Dense(dense3))
#genes_branch.add(Activation('sigmoid'))

# microRNAs
mirs_branch = Sequential()
mirs_branch.add(Dense(dense1, input_dim=genes_1.shape[1]))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
mirs_branch.add(Dense(dense2))
mirs_branch.add(Activation('relu'))
mirs_branch.add(Dropout(0.5))
#mirs_branch.add(Dense(dense3))
#mirs_branch.add(Activation('sigmoid'))

R_Q_D = Merge([mirs_branch, genes_branch], mode = 'cos') # See equation (4).
model_Rs = Sequential()
model_Rs.add(R_Q_D)
print model_Rs.output_shape
model_Rs.add(Reshape((J + 1, 1)))
print model_Rs.output_shape
weight = np.array([1]).reshape(1, 1, 1,1)
model_Rs.add(Conv1D(1, 1, border_mode = "same", input_shape = (J + 1, 1), activation = "linear", bias = False,
                       weights = [weight]))
model_Rs.add(Reshape((J + 1, )))
print model_Rs.output_shape

sgd = SGD(lr=0.00015, decay=1e-6, momentum=0.8, nesterov=True)
#model_Rs.add(Lambda(lambda x: backend.softmax(x), output_shape = (J + 1, )))
model_Rs.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics=['accuracy'])

model_Rs.fit([mirs, genes], targets,
             batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_split=0.1)

(None, 1)
(None, 1, 1)
(None, 1)
Train on 28800 samples, validate on 3200 samples
Epoch 1/200
28800/28800 [==============================] - 3s - loss: 2.4717 - acc: 0.7514 - val_loss: 3.9892 - val_acc: 0.7525
Epoch 2/200
28800/28800 [==============================] - 2s - loss: 3.9971 - acc: 0.7520 - val_loss: 3.9892 - val_acc: 0.7525
Epoch 3/200
28800/28800 [==============================] - 2s - loss: 3.9971 - acc: 0.7520 - val_loss: 3.9892 - val_acc: 0.7525
Epoch 4/200
28800/28800 [==============================] - 2s - loss: 3.9971 - acc: 0.7520 - val_loss: 3.9892 - val_acc: 0.7525
Epoch 5/200
27008/28800 [===========================>..] - ETA: 0s - loss: 3.9818 - acc: 0.7530

KeyboardInterrupt: 

In [36]:
# test
model_Rs.evaluate([mirs_t, genes_t], targets_t, batch_size=batch_size)

 5376/10000 [===============>..............] - ETA: 0s

[3.8224938488006592, 0.47339999999999999]